# explore_sentiment.py: Python script extracted from notebooks/explore_sentiment.ipynb
# Purpose: Test sentiment analysis and keyword extraction for Task 2


In [1]:
import pandas as pd
import spacy
from transformers import pipeline
from collections import Counter

c:\Users\Cyber Defense\Desktop\week2\fintech-reviews-analytics\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load cleaned reviews

In [2]:
df = pd.read_csv('../data/processed/reviews_clean.csv')
print(df.shape)
print(df.head())

(1184, 13)
                                              review  rating        date  \
0  "Why don’t your ATMs support account-to-accoun...       4  2025-06-05   
1                        what is this app problem???       1  2025-06-05   
2       the app is proactive and a good connections.       5  2025-06-05   
3    I cannot send to cbebirr app. through this app.       3  2025-06-05   
4                                               good       4  2025-06-05   

                          bank       source  \
0  Commercial Bank of Ethiopia  Google Play   
1  Commercial Bank of Ethiopia  Google Play   
2  Commercial Bank of Ethiopia  Google Play   
3  Commercial Bank of Ethiopia  Google Play   
4  Commercial Bank of Ethiopia  Google Play   

                        cleaned_review  char_count  word_count  \
0  support transfer like country south        35.0         5.0   
1                              problem         7.0         1.0   
2                      good connection        15.0 

# Initialize DistilBERT sentiment pipeline

In [3]:
sentiment_analyzer = pipeline('sentiment-analysis', 
model='distilbert-base-uncased-finetuned-sst-2-english')

Device set to use cpu



# Sample 50 reviews

In [4]:
sample_df = df.sample(1000, random_state=42)

# Apply sentiment analysis

In [5]:
def get_sentiment(text):
    result = sentiment_analyzer(text[:512])[0]  # Truncate to 512 tokens
    return result['label'], result['score']

sample_df['sentiment'] = sample_df['review'].apply(lambda x: get_sentiment(x)[0])
sample_df['sentiment_score'] = sample_df['review'].apply(lambda x: get_sentiment(x)[1])
print(sample_df[['review', 'sentiment', 'sentiment_score']].head())    

                                                 review sentiment  \
319                                                  ok  POSITIVE   
956   All-in-one finance & e-commerce super app! Pay...  POSITIVE   
1094                                  Wow Excellent app  POSITIVE   
86                                    I hate this app 😒  NEGATIVE   
990                   it is the most amazing mobile app  POSITIVE   

      sentiment_score  
319          0.999785  
956          0.994359  
1094         0.999844  
86           0.999638  
990          0.999872  


# Aggregate by bank and rating

In [6]:
sentiment_by_bank = sample_df.groupby(['bank', 'rating'])['sentiment_score'].mean().unstack()
print(sentiment_by_bank)

rating                              1         2         3         4         5
bank                                                                         
Bank of Abyssinia            0.979258  0.950136  0.960817  0.952379  0.959533
Commercial Bank of Ethiopia  0.971017  0.978796  0.974088  0.958783  0.977350
Dashen Bank                  0.993916  0.947908  0.997680  0.973680  0.983484



# Load spaCy model

In [7]:
nlp = spacy.load('en_core_web_sm')


# Function to extract keywords

In [8]:
def extract_keywords(text):
    doc = nlp(text.lower())
    keywords = [token.text for token in doc if token.pos_ in ['NOUN', 'PROPN'] or token.dep_ == 'compound']
    return keywords

# Apply to sample

In [9]:
sample_df['keywords'] = sample_df['review'].apply(extract_keywords)
print(sample_df[['review', 'keywords']].head())

                                                 review  \
319                                                  ok   
956   All-in-one finance & e-commerce super app! Pay...   
1094                                  Wow Excellent app   
86                                    I hate this app 😒   
990                   it is the most amazing mobile app   

                                    keywords  
319                                       []  
956   [finance, e, -, app, pay, bankandshop]  
1094                                   [app]  
86                                  [app, 😒]  
990                                    [app]  


# Group keywords by bank

In [10]:
keywords_by_bank = sample_df.groupby('bank')['keywords'].apply(lambda x: Counter([kw for sublist in x for kw in sublist]))
for bank, counter in keywords_by_bank.items():
    print(f"{bank}: {counter.most_common(10)}")

AttributeError: 'float' object has no attribute 'most_common'


# Example manual theme clustering

In [11]:
themes = {
    'Account Access Issues': ['login', 'password', 'authentication'],
    'Transaction Performance': ['transfer', 'payment', 'slow'],
    'User Interface': ['ui', 'design', 'navigation']
}
print('Sample Themes:', themes)

Sample Themes: {'Account Access Issues': ['login', 'password', 'authentication'], 'Transaction Performance': ['transfer', 'payment', 'slow'], 'User Interface': ['ui', 'design', 'navigation']}


In [ ]:
import pandas as pd
import oracledb
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load data (assuming Cell 2 code is reused)
dsn = oracledb.makedsn('localhost', 1521, service_name='XEPDB1')
with oracledb.connect(user='sys', password='admin', dsn=dsn, mode=oracledb.SYSDBA) as connection:
    cursor = connection.cursor()
    cursor.execute("ALTER SESSION SET CONTAINER = bank_reviews")
    df = pd.read_sql("SELECT REVIEW_ID, BANK_ID, REVIEW_TEXT, RATING, REVIEW_DATE, SOURCE, SENTIMENT_LABEL, SENTIMENT_SCORE, KEYWORDS, THEMES FROM reviews", connection)

bank_map = {1: 'Commercial Bank of Ethiopia', 2: 'Bank of Abyssinia', 3: 'Dashen Bank'}
df['bank_name'] = df['BANK_ID'].map(lambda x: bank_map.get(x) if pd.notnull(x) else None)
df['REVIEW_DATE'] = pd.to_datetime(df['REVIEW_DATE'])

# Figure 2: Sentiment Counts by Bank
sentiment_counts = df.groupby(['bank_name', 'SENTIMENT_LABEL']).size().unstack(fill_value=0)
plt.figure(figsize=(10, 6))
sentiment_counts.plot(kind='bar')
plt.title('Sentiment Counts by Bank')
plt.xlabel('Bank Name')
plt.ylabel('Count')
plt.legend(title='Sentiment Label')
plt.tight_layout()
plt.savefig('../data/visualizations/sentiment_counts.png')
plt.close()

# Figure 3: Theme Frequencies Across All Banks
def extract_theme_counts(df):
    theme_counts = {}
    for index, row in df.iterrows():
        themes = eval(row['THEMES']) if row['THEMES'] else []
        for theme, _ in themes:
            theme_counts[theme] = theme_counts.get(theme, 0) + 1
    return pd.Series(theme_counts)

all_themes = extract_theme_counts(df)
plt.figure(figsize=(10, 6))
all_themes.plot(kind='bar', stacked=True)
plt.title('Theme Frequencies Across All Banks')
plt.xlabel('Theme')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig('../data/visualizations/theme_frequencies.png')
plt.close()

# Figure 6: Average Rating Trends by Bank
df_grouped = df.groupby([pd.Grouper(key='REVIEW_DATE', freq='M'), 'bank_name'])['RATING'].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_grouped, x='REVIEW_DATE', y='RATING', hue='bank_name', marker='o')
plt.title('Average Rating Trends by Bank')
plt.xlabel('Date')
plt.ylabel('Average Rating')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../data/visualizations/rating_trends.png')
plt.close()

# Figure 7: Keyword Correlation Heatmap
keywords = df['KEYWORDS'].apply(lambda x: eval(x) if x else [])
all_keywords = [kw for sublist in keywords for kw in sublist]
keyword_correlations = pd.get_dummies(pd.Series(all_keywords)).corr()
plt.figure(figsize=(10, 6))
sns.heatmap(keyword_correlations, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Keyword Correlation Heatmap')
plt.tight_layout()
plt.savefig('../data/visualizations/keyword_correlation.png')
plt.close()

C:\Users\Cyber Defense\AppData\Local\Temp\ipykernel_20912\3640742976.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT REVIEW_ID, BANK_ID, REVIEW_TEXT, RATING, REVIEW_DATE, SOURCE, SENTIMENT_LABEL, SENTIMENT_SCORE, KEYWORDS, THEMES FROM reviews", connection)
C:\Users\Cyber Defense\AppData\Local\Temp\ipykernel_20912\3640742976.py:50: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_grouped = df.groupby([pd.Grouper(key='REVIEW_DATE', freq='M'), 'bank_name'])['RATING'].mean().reset_index()
